# 1. Setup

## 1.1. Import statements

In [1]:
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import matplotlib.pyplot as plt
from torchinfo import summary
from tqdm import tqdm
from PIL import Image
from torch.utils.data import DataLoader, random_split
from torch.optim.swa_utils import AveragedModel, SWALR, update_bn
from torch.amp import autocast, GradScaler
from torchvision.transforms import v2
from torchvision.datasets import ImageFolder

## 1.2. Device configuration

In [3]:
def get_device() -> torch.device:
    """Get the best available device for PyTorch."""
    if torch.cuda.is_available():
        device = "cuda"

        # The flag below controls whether to allow TF32 on matmul.
        torch.backends.cuda.matmul.allow_tf32 = True
        # The flag below controls whether to allow TF32 on cuDNN.
        torch.backends.cudnn.allow_tf16 = True

        # Print GPU info
        print(f"Using CUDA device: {torch.cuda.get_device_name(0)}")
        
        # Set up GPU memory management
        memory_limit_mb = 4095.5  # Adjust as needed

        total_memory = torch.cuda.get_device_properties(0).total_memory

        memory_limit = memory_limit_mb * 1024 ** 2
        memory_fraction = memory_limit / total_memory

        torch.cuda.set_per_process_memory_fraction(memory_fraction, device=0)

        print(f"Set GPU memory fraction to {memory_fraction:.2%}")

        # Ensure memory is allocated
        torch.cuda.empty_cache()
    elif torch.backends.mps.is_available():
        device = "mps"
        print("Using Apple Silicon MPS device")
    else:
        device = "cpu"
        print("Using CPU device")
    
    return torch.device(device)

In [4]:
torch.set_float32_matmul_precision('high')
device = get_device()

Using CUDA device: NVIDIA GeForce RTX 3050 Ti Laptop GPU
Set GPU memory fraction to 100.00%


In [ ]:
pin_memory = True if device.type == 'cuda' else False
pin_memory_device = 'cuda' if device.type == 'cuda' else ''

# 2. Load & transform data

## 2.1. Normalize the dataset

In [ ]:
# First, create transforms without normalization to calculate dataset statistics
initial_transforms = v2.Compose([
    v2.Resize((360, 640)),
    v2.Grayscale(num_output_channels=3),
    v2.ToImage(),
    v2.ToDtype(torch.float16, scale=True)
])

In [ ]:
label_transforms = v2.Lambda(
    lambda y: torch.zeros(15, dtype=torch.float16).scatter_(dim=0, index=torch.tensor(y, dtype=torch.long), value=1)
)

In [ ]:
# Create temporary dataset to calculate mean and std
temp_dataset = ImageFolder(root='../data/data_pool', transform=initial_transforms, target_transform=label_transforms)

temp_loader = DataLoader(temp_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=pin_memory, pin_memory_device=pin_memory_device)

In [ ]:
calc_stats = False

if calc_stats:
    print(f"Computing dataset statistics using device: {device}")
    print(f"Number of images to process: {len(temp_dataset)}")

    channels_sum = torch.zeros(3, device=device)
    channels_sqrd_sum = torch.zeros(3, device=device)
    num_batches = 0
    start_time = time.time()

    for batch_idx, (data, _) in enumerate(tqdm(temp_loader, desc="Computing mean/std")):
        data = data.to(device)  # Add non_blocking=True
        with autocast():
            channels_sum += torch.mean(data, dim=[0, 2, 3])
            channels_sqrd_sum += torch.mean(data ** 2, dim=[0, 2, 3])
        num_batches += 1
        
        if batch_idx % 10 == 0:
            batch_time = time.time() - start_time
            print(f"\nProcessed {batch_idx * temp_loader.batch_size} images in {batch_time:.2f}s")

    mean = channels_sum / num_batches
    std = torch.sqrt((channels_sqrd_sum / num_batches) - (mean ** 2))

    mean = mean.cpu().tolist()
    std = std.cpu().tolist()

    total_time = time.time() - start_time
    print(f"\nTotal processing time: {total_time:.2f} seconds")
    print(f"Dataset mean: {mean}")
    print(f"Dataset std: {std}")
else:
    mean=[0.36145609617233276, 0.3521592915058136, 0.3483520746231079]
    std=[0.20100851356983185, 0.21270669996738434, 0.254261314868927]

## 2.2. Define transforms

In [ ]:
train_transforms = v2.Compose([
    v2.Resize((360, 640)),
    v2.Grayscale(num_output_channels=3),
    v2.RandomHorizontalFlip(p=0.2),
    v2.RandomVerticalFlip(p=0.2),
    # New subtle augmentations
    v2.ColorJitter(
        brightness=0.1,  # Subtle brightness changes for different lighting
        contrast=0.1,    # Subtle contrast changes for different cameras/lighting
        saturation=0.1,  # Subtle color saturation changes
    ),
    v2.RandomAdjustSharpness(sharpness_factor=1.1, p=1.0),  # Subtle sharpness changes
    # Small perspective changes to simulate different card angles
    v2.RandomPerspective(
        distortion_scale=0.15,  # Keep perspective changes minimal
        p=0.2                   # Apply only 30% of the time
    ),
    v2.ToImage(),
    v2.ToDtype(torch.float16, scale=True),
    v2.Normalize(mean=mean, std=std)
])

In [ ]:
val_transforms = v2.Compose([
    v2.Resize((360, 640)),
    v2.Grayscale(num_output_channels=3),
    v2.RandomAdjustSharpness(sharpness_factor=1.1, p=1.0),  # Subtle sharpness changes
    v2.ToImage(),
    v2.ToDtype(torch.float16, scale=True),
    v2.Normalize(mean=mean, std=std),
])

# 3. Partition dataset

In [ ]:
# Load the dataset with appropriate transforms
full_dataset = ImageFolder(root='../data/data_pool', transform=None)

In [ ]:
# Define split ratios
train_ratio = 0.75
val_ratio = 0.15
test_ratio = 0.10

# calculate lengths
total_size = len(full_dataset)
train_size = int(train_ratio * total_size)
val_size = int(val_ratio * total_size)
test_size = total_size - train_size - val_size

In [ ]:
# Create train/val/test datasets with appropriate transforms
train_data, val_data, test_data = random_split(
    full_dataset, 
    [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)  # For reproducibility
)

In [ ]:
# Directly override transforms
train_data.dataset.transform = train_transforms
train_data.dataset.target_transform = label_transforms

val_data.dataset.transform = val_transforms 
val_data.dataset.target_transform = label_transforms

test_data.dataset.transform = val_transforms
test_data.dataset.target_transform = label_transforms

In [ ]:
# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=16, shuffle=True, num_workers=4, pin_memory=pin_memory, pin_memory_device=pin_memory_device)

val_loader = DataLoader(val_data, batch_size=16, shuffle=False, num_workers=4, pin_memory=pin_memory, pin_memory_device=pin_memory_device)  

test_loader = DataLoader(test_data, batch_size=16, shuffle=False, num_workers=4, pin_memory=pin_memory, pin_memory_device=pin_memory_device)

# 4. Building the convolutional neural network

In [ ]:
# Input shape constants
IMG_WIDTH = 360
IMG_HEIGHT = 640
IMG_CHANNELS = 3

In [ ]:
class UnoSymbolClassifier(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.conv_block1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, bias=False, padding=1),  # Output: 16 x H x W
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 16, kernel_size=3, padding=1, bias=False),  # Output: 16 x H x W
            nn.BatchNorm2d(16),
            nn.ReLU(),
        )
        
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1, bias=False),  # Output: 32 x H/2 x W/2
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1, bias=False),  # Output: 32 x H/2 x W/2
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Downsample by 2
        )
        
        self.conv_block3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1, bias=False),  # Output: 64 x H/4 x W/4
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1, bias=False),  # Output: 64 x H/4 x W/4
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Downsample by 2
        )
        
        self.conv_block4 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1, bias=False),  # Output: 128 x H/8 x W/8
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1, bias=False),  # Output: 128 x H/8 x W/8
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Downsample by 2
        )
        
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))  # Output: 128 x 1 x 1
        
        self.fc = nn.Sequential(
            nn.Flatten(),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(32, 16),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(16, 15)  # Assuming 15 classes
        )

    def forward(self, x) -> torch.utils.data.Dataset:
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)
        x = self.global_avg_pool(x)
        x = self.fc(x)

        return x

## 4.2. Create the model

In [ ]:
model = UnoSymbolClassifier()
model.to(device)
model = torch.compile(model, backend="inductor")

In [ ]:
summary(model, input_size=(32, IMG_CHANNELS, IMG_HEIGHT, IMG_WIDTH))

# 5. Optimising model parameters

## 5.1. Learning parameters

In [ ]:
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 5e-3
EPOCHS = 25

## 5.2. Optimizer & cost function

In [ ]:
loss_fn = nn.CrossEntropyLoss()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY, fused=True)

In [ ]:
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.25, patience=5)

# 6. Training the model

## 6.1. Define train function

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    scaler = GradScaler()
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()

    training_loss = 0.0
    correct = 0
    total = 0

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device, non_blocking=True), y.to(device, non_blocking=True)  # Move data to device

        optimizer.zero_grad()

        with autocast():
            # Compute prediction and loss
            pred = model(X)
            loss = loss_fn(pred, y)

        # Backpropagation
        scaler.scale(loss.backward())
        scaler.scale(optimizer.step())
        scaler.update()

        training_loss += loss.item() * X.size(0)
        correct += (pred.argmax(1) == y).type(torch.float16).sum().item()
        total += y.size(0)

        if batch % 100 == 0:
            loss_item = loss.item()
            current = batch * len(X)
            print(f"loss: {loss_item:>7f}  [{current:>5d}/{size:>5d}]")

    avg_loss = training_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

## 6.2. Define validate & test function

In [ ]:
def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device, non_blocking=True), y.to(device, non_blocking=True)  # Move data to device

            with autocast():
                pred = model(X)
                loss = loss_fn(pred, y)

            test_loss += loss.item() * X.size(0)
            correct += (pred.argmax(1) == y).type(torch.float16).sum().item()
            total += y.size(0)

    avg_loss = test_loss / total
    accuracy = correct / total
    print(f"Avg loss: {avg_loss:>8f}, Accuracy: {(100*accuracy):>0.1f}%\n")

    return avg_loss, accuracy

## 6.3. Define overfitting function

In [ ]:
def check_overfitting(train_loss, val_loss, train_acc, val_acc, threshold=0.1):
    loss_gap = abs(train_loss - val_loss)
    acc_gap = abs(train_acc - val_acc)
    
    is_overfitting = (loss_gap > threshold) and (train_acc > val_acc + threshold)
    
    if is_overfitting:
        print(f"Warning: Possible overfitting detected")
        print(f"Loss gap: {loss_gap:.4f}, Accuracy gap: {acc_gap:.4f}")
    
    return is_overfitting

## 6.4. Training loop

In [ ]:
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []
epoch_times = []

best_val_loss = float('inf')
best_model_metrics = None
stopped_early = False
is_overfitting = 0
patience = 10  # Number of epochs with no improvement after which training will be stopped
total_start_time = time.time()

In [ ]:
for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}\n-------------------------------")
    epoch_start_time = time.time()

    train_loss, train_accuracy = train_loop(train_loader, model, loss_fn, optimizer)

    scheduler.step(train_loss)  # Use regular scheduler before SWA
        
    val_loss, val_accuracy = test_loop(val_loader, model, loss_fn)

    epoch_time = time.time() - epoch_start_time
    epoch_times.append(epoch_time)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)

    print(f"Epoch {epoch+1} completed in {epoch_time:.2f} seconds")
    print(f"Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}\n")

    # Increment overfitting counter if overfitting detected
    if check_overfitting(train_loss, val_loss, train_accuracy, val_accuracy):
        is_overfitting += 1
    else:
        is_overfitting = 0

    # Check all conditions
    accuracy_gap = abs(train_accuracy - val_accuracy)
    conditions_met = (
        train_loss > 0.13 and
        val_loss > 0.13 and
        train_accuracy < 0.99 and
        val_accuracy < 0.99 and
        accuracy_gap < 0.03
    )

    # Save model if conditions are met and validation loss improved
    if conditions_met and val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_metrics = {
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'train_accuracy': train_accuracy,
            'val_accuracy': val_accuracy
        }
        torch.save(model.state_dict(), '../data/models/best_model.pth')

    # Stop if overfitting persists for multiple epochs
    if is_overfitting >= patience:
        print(f"Early stopping triggered due to persistent overfitting.")
        stopped_early = True
        break

total_training_time = time.time() - total_start_time

# Save both models
torch.save(model, '../data/models/final_model.pth')

if best_model_metrics:
    print("\nBest model saved with metrics:")
    for key, value in best_model_metrics.items():
        print(f"{key}: {value}")

print(f"\nTraining complete in {total_training_time:.2f} seconds")
print("\n-------------------------------\nDone!")

# 7. Plot model metrics

In [ ]:
epochs_range = range(1, len(train_losses) + 1)

## 7.1. Loss graph

In [ ]:
# Plot Losses
plt.figure(figsize=(10, 5))
plt.plot(epochs_range, train_losses, label='Training Loss')
plt.plot(epochs_range, val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

## 7.2. Accuracy graph

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(epochs_range, train_accuracies, label='Training Accuracy')
plt.plot(epochs_range, val_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

## 7.3. Epoch duration graph

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(epochs_range, epoch_times, label='Time per Epoch')
plt.xlabel('Epochs')
plt.ylabel('Time (seconds)')
plt.title('Time Taken per Epoch')
plt.legend()
plt.show()

# 8. Test the model

## 8.1. Create & load model

In [ ]:
model = UnoSymbolClassifier()
model.to(device)
model = torch.compile(model, backend="inductor")

In [ ]:
stopped_early = False

# Load the best model (if saved during early stopping)
if stopped_early:
    model.load_state_dict(torch.load('../data/models/best_symbol_classifier.pth'))
else:
    model = torch.load('../data/models/full_symbol_classifier.pth')

## 8.2. Test model on the dataset

In [ ]:
model.eval()

print("Test Results on the Test Set:")
test_loop(test_loader, model, loss_fn)

## 8.3. Test model on own image

In [ ]:
image_transform = v2.Compose([
    v2.Grayscale(num_output_channels=3),
    v2.ToImage(),
    v2.ToDtype(torch.float16, scale=True),
])

In [ ]:
# Load and preprocess the image
image_path = '../notebooks/Yellow_Draw_2.jpg' 
image = Image.open(image_path).convert('RGB')
input_tensor = image_transform(image)
input_tensor = input_tensor.unsqueeze(0)  # Add batch dimension
input_tensor = input_tensor.to(device)

In [ ]:
# Make prediction
with torch.no_grad():
    with autocast():  # Add autocast here for consistent mixed precision inference
        output = model(input_tensor)
    predicted_class = output.argmax(dim=1).item()

In [ ]:
# Get class names
class_names = full_dataset.classes
predicted_label = class_names[predicted_class]
print(f"Predicted class: {predicted_label}")